In [ ]:
# example of a dcgan on cifar10
from numpy import expand_dims
from numpy import zeros
from numpy import ones
from numpy import vstack
from numpy.random import randn
from numpy.random import randint
from tensorflow.keras.datasets.cifar10 import load_data
from tensorflow.keras.optimizers import Adam
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Reshape
from keras.layers import Flatten
from keras.layers import Conv2D
from keras.layers import Conv2DTranspose
from keras.layers import LeakyReLU
from keras.layers import Dropout
from matplotlib import pyplot
import pandas as pd
import numpy as np

In [ ]:
# define the standalone discriminator model
def define_discriminator(in_shape=(32,32,3)):
	model = Sequential()
	# normal
	model.add(Conv2D(64, (3,3), padding='same', input_shape=in_shape))
	model.add(LeakyReLU(alpha=0.2))
	# downsample
	model.add(Conv2D(128, (3,3), strides=(2,2), padding='same'))
	model.add(LeakyReLU(alpha=0.2))
	# downsample
	model.add(Conv2D(128, (3,3), strides=(2,2), padding='same'))
	model.add(LeakyReLU(alpha=0.2))
	# downsample
	model.add(Conv2D(256, (3,3), strides=(2,2), padding='same'))
	model.add(LeakyReLU(alpha=0.2))
	# classifier
	model.add(Flatten())
	model.add(Dropout(0.4))
	model.add(Dense(1, activation='sigmoid'))
	# compile model
	opt = Adam(lr=0.0002, beta_1=0.5)
	model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
	return model

In [ ]:
# define the standalone generator model
def define_generator(latent_dim):
	model = Sequential()
	# foundation for 4x4 image
	n_nodes = 256 * 4 * 4
	model.add(Dense(n_nodes, input_dim=latent_dim))
	model.add(LeakyReLU(alpha=0.2))
	model.add(Reshape((4, 4, 256)))
	# upsample to 8x8
	model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
	model.add(LeakyReLU(alpha=0.2))
	# upsample to 16x16
	model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
	model.add(LeakyReLU(alpha=0.2))
	# upsample to 32x32
	model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
	model.add(LeakyReLU(alpha=0.2))
	# output layer
	model.add(Conv2D(3, (3,3), activation='tanh', padding='same'))
	return model

In [ ]:
# define the combined generator and discriminator model, for updating the generator
def define_gan(g_model, d_model):
	# make weights in the discriminator not trainable
	d_model.trainable = False
	# connect them
	model = Sequential()
	# add generator
	model.add(g_model)
	# add the discriminator
	model.add(d_model)
	# compile model
	opt = Adam(lr=0.0002, beta_1=0.5)
	model.compile(loss='binary_crossentropy', optimizer=opt)
	return model

def load_data(n_samples = 1000, dimen =32*32):
    data_user = pd.read_csv('../data/users_cl/data_user_002.csv')
    data_user['hour'] = pd.to_datetime(data_user['time']).dt.hour
    data_formatted = data_user[['hour','lat','lon']]
    min_1, min_2, min_3 = data_formatted.min(axis=0)
    max_1, max_2, max_3 = data_formatted.max(axis=0)
    mins = [min_1,min_2,min_3]
    maxs = [max_1,max_2,max_3]
    data_group = pd.DataFrame()
    for i in range(n_samples):
        data_group = pd.concat([data_group,data_formatted.sample(dimen).reshape(-1,1)])
    return data_group, mins,maxs

In [ ]:
32

In [ ]:
# load and prepare cifar10 training images
def load_real_samples(n_traces = 1000, n_points = 32*32):
	from sklearn.preprocessing import MinMaxScaler
	data_user = pd.read_csv('../data/users/data_user_002.csv')
	data_user['hour'] = pd.to_datetime(data_user['time']).dt.hour
	data_formatted = data_user[['hour','lat','lon']]
	scaler = MinMaxScaler()
	data_formatted = pd.DataFrame(scaler.fit_transform(data_formatted))
	data_formatted.columns = ['hour','lat','lon']
	#min_1, min_2, min_3 = data_formatted.min(axis=0)
	#max_1, max_2, max_3 = data_formatted.max(axis=0)
	trainX = pd.DataFrame()
	for i in range(n_traces):
		trainX = pd.concat([trainX,data_formatted.sample(n_points)])
	trainX = trainX.values.reshape((n_traces,32,32,3))
	X = trainX.astype('float32')
	

	return X, scaler


	# load cifar10 dataset
	#trainX = load_data()
	# convert from unsigned ints to floats
	#X = trainX.astype('float32')
	# scale from [0,255] to [-1,1]
	#X = (X - 127.5) / 127.5
	#return X

In [ ]:
# select real samples
def generate_real_samples(dataset, n_samples):
	# choose random instances
	ix = randint(0, dataset.shape[0], n_samples)
	# retrieve selected images
	X = dataset[ix]
	# generate 'real' class labels (1)
	y = ones((n_samples, 1))
	return X, y

In [ ]:
# generate points in latent space as input for the generator
def generate_latent_points(latent_dim, n_samples):
	# generate points in the latent space
	x_input = randn(latent_dim * n_samples)
	# reshape into a batch of inputs for the network
	x_input = x_input.reshape(n_samples, latent_dim)
	return x_input

In [ ]:
# use the generator to generate n fake examples, with class labels
def generate_fake_samples(g_model, latent_dim, n_samples):
	# generate points in latent space
	x_input = generate_latent_points(latent_dim, n_samples)
	# predict outputs
	X = g_model.predict(x_input)
	# create 'fake' class labels (0)
	y = zeros((n_samples, 1))
	return X, y

In [ ]:
# create and save a plot of generated images
def save_plot(examples,real, epoch, n=7):
	# scale from [-1,1] to [0,1]
	#examples = (examples + 1) / 2.0
	# plot images
	#for i in range(n * n):
		# define subplot
		#pyplot.subplot(n, n, 1 + i)
		# turn off axis
		#pyplot.axis('off')
		# plot raw pixel data
		#print(examples[i].reshape((32*32,3))[:,1].shape)
	idx = np.random.randint(100, size=1)[0]
	fake_data = examples[idx]
	real_data = real[idx]
	fig = pyplot.figure(figsize = (7, 9))
	grid = pyplot.GridSpec(1, 1, wspace =0.5, hspace = 1.5, figure=fig)

	g1 = pyplot.subplot(grid[0, 0]) # Dimension 1
	#g2 = pyplot.subplot(grid[0, 1]) # Dimension 2

	# Generación de Latitud        
	g1.scatter(x =fake_data.reshape((32*32,3))[:,1], y =fake_data.reshape((32*32,3))[:,2], alpha=0.3, label='fake')
	g1.scatter(x =real_data.reshape((32*32,3))[:,1], y =real_data.reshape((32*32,3))[:,2], alpha=0.3, label='real')
	g1.set_title('Combo: Latitud vs Longitud')
	g1.set_ylabel('Longitud')
	g1.set_xlabel('Latitud')
	pyplot.legend()
	#g1.text(0.05,0.95,textstr,transform=g4.transAxes,fontsize=14,verticalalignment="top",bbox=props,)


	#pyplot.scatter(x =examples[0].reshape((32*32,3))[:,1], y =examples[0].reshape((32*32,3))[:,2])
	pyplot.show()
	# save plot to file
	#filename = 'generated_plot_e%03d.png' % (epoch+1)
	#pyplot.savefig(filename)
	#pyplot.close()

In [ ]:
# evaluate the discriminator, plot generated images, save generator model
def summarize_performance(epoch, g_model, d_model, dataset, latent_dim, n_samples=100):
	# prepare real samples
	X_real, y_real = generate_real_samples(dataset, n_samples)
	# evaluate discriminator on real examples
	_, acc_real = d_model.evaluate(X_real, y_real, verbose=0)
	# prepare fake examples
	x_fake, y_fake = generate_fake_samples(g_model, latent_dim, n_samples)
	# evaluate discriminator on fake examples
	_, acc_fake = d_model.evaluate(x_fake, y_fake, verbose=0)
	# summarize discriminator performance
	print('>Accuracy real: %.0f%%, fake: %.0f%%' % (acc_real*100, acc_fake*100))
	# save plot
	save_plot(x_fake,X_real, epoch)
	# save the generator model tile file
	filename = 'generator_model_%03d.h5' % (epoch+1)
	g_model.save(filename)

In [ ]:
# train the generator and discriminator
def train(g_model, d_model, gan_model, dataset, latent_dim, n_epochs=1000, n_batch=128):
	bat_per_epo = int(dataset.shape[0] / n_batch)
	half_batch = int(n_batch / 2)
	# manually enumerate epochs
	for i in range(n_epochs):
		# enumerate batches over the training set
		for j in range(bat_per_epo):
			# get randomly selected 'real' samples
			X_real, y_real = generate_real_samples(dataset, half_batch)
			# update discriminator model weights
			d_loss1, _ = d_model.train_on_batch(X_real, y_real)
			# generate 'fake' examples
			X_fake, y_fake = generate_fake_samples(g_model, latent_dim, half_batch)
			# update discriminator model weights
			d_loss2, _ = d_model.train_on_batch(X_fake, y_fake)
			# prepare points in latent space as input for the generator
			X_gan = generate_latent_points(latent_dim, n_batch)
			# create inverted labels for the fake samples
			y_gan = ones((n_batch, 1))
			# update the generator via the discriminator's error
			g_loss = gan_model.train_on_batch(X_gan, y_gan)
			# summarize loss on this batch
			print('>%d, %d/%d, d1=%.3f, d2=%.3f g=%.3f' %
				(i+1, j+1, bat_per_epo, d_loss1, d_loss2, g_loss))
		# evaluate the model performance, sometimes
		if (i+1) % 50 == 0:
			summarize_performance(i, g_model, d_model, dataset, latent_dim)

In [ ]:
# size of the latent space
latent_dim = 100
# create the discriminator
d_model = define_discriminator()

In [ ]:
# create the generator
g_model = define_generator(latent_dim)

In [ ]:
# create the gan
gan_model = define_gan(g_model, d_model)

In [ ]:
# load image data
dataset, scaler = load_real_samples()

In [ ]:
dataset

In [ ]:
dataset.shape

In [ ]:

# train model
train(g_model, d_model, gan_model, dataset, latent_dim)